# Tugas Besar IF3170
## Aplikasi Web Prediksi Income Per Tahun

### Nama Kelompok : 49
### Anggota :
    1. Finiko Kasula Novenda - 13515029
    2. M. Dicky Andika Putra - 13515044
    3. Vincent Hendryanto Halim - 13515089
    4. Mikhael Artur Darmakesuma - 13515099
    5. William - 13515144
### Hasil Analisis Data
#### Jenis data
Data yang tersedia dapat diklasifikasikan menjadi 2 jenis, yaitu data kontinu dan data kategorial.
- Kolom yang mengandung data kontinu antara lain: age, fnlwgt,education-num, capital-gain, capital-loss, hours-per-week
- Kolom yang mengandung data kategorial antara lain: workclass,education,marital-status,occupation, relationship, race, sex, native-country

#### Penanganan data tidak lengkap
Dari data awal, masih ada data yang mengandung informasi yang tidak lengkap, ditandai dengan karakter '?'
Data yang hilang ini kami ubah menjadi modus keseluruhan data. Data yang hilang hanya terdapat pada kolom data kategorial, dan data kategorial tidak dapat dihitung mean, sehingga dipilih modus.
#### Perubahan menjadi data numerik
Classifier yang tersedia pada scikit, baik KNN, ID3, MLP maupun NaiveBayes hanya dapat menangani data bersifat numerik.
Untuk itu data-data yang bersifat kategorial perlu diubah. Data kategorial ini diubah menggunakan fitur get_dummies pada panda. Dengan get_dummies ini, akan terbentuk kolom baru sebesar sparsity suatu kolom kategorial. Misalkan kolom "Sex" memiliki nilai Male atau Female. Setelah diubah, kolom Sex akan berubah menjadi 2 kolom baru yaitu kolom Sex_Male, dan kolom Sex_Female. Jika sebelumnya kolom Sex bernilai "Male", setelah perubahan kolom Sex_Male akan bernilai 1 dan kolom Sex_Female bernilai 0.
#### Ukuran Kinerja
Kinerja pada tugas kali ini diukur dengan menggunakan fungsi bawaan sklearn yaitu tree_model.score yang menghitung akurasi dari model terhadap data tes yang diberikan. Akurasi dihitung berdasarkan data yang berhasil diprediksi dengan benar dibanding total semua data. <i>Accuracy</i> = (Tp + Tn)/(Tp + Fp + Tn + Fn).

In [41]:
# Library Import
import pandas as pd
from pandas import DataFrame
import graphviz
from sklearn import preprocessing
import pickle

# Algorithm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

## 1. Preprocessing Data

In [42]:
atributeNames = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","target"]
income = pd.read_csv('data/CensusIncome.data.csv', header=None, names = atributeNames, sep = ",\s", engine="python", na_values="?");

### Mengubah NaN menjadi Most Frequent

In [43]:
income = income.apply(lambda x:x.fillna(x.value_counts().index[0]))

### Panda get_dummy

In [44]:
incomeFinal = pd.get_dummies(income, columns=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"])
x = incomeFinal[[i for i in list(incomeFinal.columns) if i != 'target']].values
y = incomeFinal['target'].values

## 2. Pemilihan Algoritma

### 2.1 KNN

In [6]:
nbrs = KNeighborsClassifier(n_neighbors=15)
score = cross_val_score(nbrs, x, y, cv=10)
print("Score: ", score.mean())

Score:  0.796689345722


In [7]:
y_pred = cross_val_predict(nbrs,x,y,cv=10)
print(confusion_matrix(y,y_pred))

[[24124   596]
 [ 6024  1817]]


### 2.2 MLP

In [8]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(70,25), random_state=1)

print ("Score : ", cross_val_score(mlp, x, y, cv=10).mean())

Score :  0.759190448997


In [9]:
y_pred = cross_val_predict(mlp, x, y, cv=10)
print(confusion_matrix(y,y_pred))

[[24720     0]
 [ 7841     0]]


### 2.3 Naive Bayes

In [41]:
foldNaiveBayes = GaussianNB()
score = cross_val_score(foldNaiveBayes, x, y, cv = 10)
print("Score: ", score.mean())

Score:  0.795307347848


In [42]:
y_pred = cross_val_predict(foldNaiveBayes, x, y, cv = 10)
print(confusion_matrix(y,y_pred))

[[23459  1261]
 [ 5404  2437]]


### 2.4 Decision Tree ID3

In [10]:
dt = tree.DecisionTreeClassifier(criterion='entropy')
score = cross_val_score(dt, x, y, cv = 10)
print("Score: ", score.mean())

Score:  0.820798022253


In [11]:
y_pred = cross_val_predict(dt, x, y, cv = 10)
print(confusion_matrix(y,y_pred))

[[21681  3039]
 [ 2838  5003]]


## 3. Full Training

Dipilih algoritma Decision Tree ID3 Karena menghasilkan nilai akurasi yang paling baik

In [45]:
dt_out = tree.DecisionTreeClassifier(criterion='entropy', max_features = 53)
dt_out = dt_out.fit(x,y)
#Menyimpan model
pickle.dump(dt_out,open("out.pkl", "wb+"))

In [46]:
test_data = pd.read_csv('data/CensusIncome.test.csv', header=None, names = atributeNames, sep = ",\s", engine="python", na_values="?", comment="|");
test_data = test_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
test_data = pd.get_dummies(test_data, columns=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"])

test_dataframe = pd.DataFrame(columns = incomeFinal.columns)
test_dataframe.append(test_data)

test_dataframe = pd.concat([test_dataframe, test_data], axis=0)
test_dataframe = test_dataframe.fillna(0)

x_test = test_dataframe[[i for i in list(test_dataframe.columns) if i != 'target']].values
y_test = pd.DataFrame(test_dataframe['target'].values)

In [47]:
#Melakukan load model
tree_model = pickle.load(open("out.pkl","rb+"))

score = tree_model.score(x_test, y_test)
print("Score : %f" % (score*100))

Score : 64.807763


In [48]:
y_pred = tree_model.predict(x_test)
print(confusion_matrix(y_test,y_pred))

[[10405  2031]
 [ 3699   147]]
